# Narrative Style and Spread of Misinformation on Twitter

## Importing necessary Libraries

In [ ]:
# imports - native Python
import collections
import csv
import os
import re
# imports - 3rd party
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# installs from 🤗
! pip install transformers
! pip install datasets
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict

## Loading the Dataset

- Initialize a Dictionary: A dictionary with keys 'text' and 'label' is set up to store dataset contents.
- File Selection: The script identifies TSV files in the current directory with names ending in 'dataset.tsv'.
- File Reading: Each selected TSV file is read as a dictionary with tab-separated values, using 'latin-1' encoding.
- Data Extraction: Non-empty 'Message' field values and corresponding 'Narrative (1)' integer labels are extracted and added to the dataset dictionary.
- Dataset Conversion: The filled dataset dictionary is converted into a Hugging Face Dataset object using Dataset.from_dict().


In [ ]:
dataset_dict = {'text':[], 'label':[]} 
for f in os.listdir():
  # use all .tsv files that have been loaded
  if f.endswith('dataset.tsv'):
    with open(f,encoding='latin-1') as tsv_file:
        reader = csv.DictReader(tsv_file, dialect='excel-tab')
        for line in reader:
            text = line['Message']
        # a few of the Message fields are empty, so we should skip those ones
            if text!=None and text.strip()!="":
                dataset_dict['text'].append(text)
                dataset_dict['label'].append(int(line['Narrative (1)']))
# huggingface function to convert from dict to their Dataset object
# which will work nicely with their model trainer
ds = Dataset.from_dict(dataset_dict)

## Split The data

- **Initial Split:** The dataset ds is split into two parts:
    - 80% for training (train_testvalid['train'])
    - 20% reserved for further splitting into validation and test sets (train_testvalid['test'])
- **Secondary Split:** The 20% test set from the initial split is further divided evenly into:
    - 10% for validation (test_valid['train'])
    - 10% for the final test set (test_valid['test'])
- **Dataset Dictionary Creation:** A DatasetDict object is constructed to organize the splits into a single dataset with three keys:
    - 'train': The training set (80% of the original dataset)
    - 'test': The test set (10% of the original dataset)
    - 'valid': The validation set (10% of the original dataset)
- **Data Integrity Check:** The code performs an assertion to verify that none of the text entries in any of the splits are None. This acts as a quality control step to confirm that the dataset was filtered correctly in earlier steps, removing any None values.

In [ ]:
train_testvalid = ds.train_test_split(test_size=0.2)
# then split the 20 into 10-10 validation and test
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# finally, make the full dataset the 80-10-10 split as a DatasetDict object
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
# quick check (if this doesn't pass, will get an error in the tokenization)
# makes sure we filtered the data correcly at the beginning and removed None
for split in train_test_valid_dataset.keys():
    assert not any([x==None for x in train_test_valid_dataset[split]['text']])

## Tokenization

This snippet of code sets up tokenization for a dataset using a pretrained tokenizer from the Hugging Face transformers library, specifically the bert-base-uncased model. It creates an instance of a tokenizer that is pre-trained on the 'bert-base-uncased' model. The 'uncased' part means that the tokenizer does not distinguish between uppercase and lowercase letters.

A function named *tokenize* is defined to handle the tokenization process. It takes input data ('examples') and a key ('textfield') that specifies which field of the data to tokenize. Inside the function:
- The tokenizer converts the text into tokens, adds necessary padding to each text entry to reach the maximum length required by the model, and truncates any text that exceeds this maximum length.
- The map function from the Hugging Face datasets library is used to apply the tokenize function to all entries in the 'train_test_valid_dataset'.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# simple wrapper
def tokenize(examples, textfield="text"):
    return tokenizer(examples[textfield], padding="max_length", truncation=True)

# batch tokenization
tokenized_datasets = train_test_valid_dataset.map(tokenize, batched=True)

# Model

- A *DataCollatorWithPadding* object is created and is given the previously initialized tokenizer. This data collator is responsible for dynamically padding the tokenized inputs to the longest sequence in a batch, ensuring that all sequences in a batch have the same length. This step is necessary because models like BERT require uniform input shapes. However, since texts naturally have varying lengths, padding must be done at the batch level for efficiency. 
- We then load a pretrained BERT model with a sequence classification head on top (a single linear layer used for classification tasks) using 'AutoModelForSequenceClassification'.
- The 'num_labels=2' parameter indicates that the model is being set up for a binary classification task (e.g., positive vs. negative sentiment analysis).

In [ ]:
# Setup collation
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

## Compute Metrics and Training Arguments

The compute_metrics function calculates evaluation metrics for a binary classification task by comparing the true labels of the dataset with the predictions made by a model. Specifically, it computes the following metrics: *precision*, *recall*, *f1-score* and *accuracy*

In [ ]:
# using sklearn to compute precision, recall, f1, and accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

The code snippet below is setting up a *Trainer* object from the Hugging Face transformers library. This Trainer is configured to fine-tune the previously initialized BERT model for a sequence classification task. *TrainingArguments* is a configuration class that includes all the hyperparameters for training. 
The trainer object is fully configured to start training the model. It contains all necessary information to fine-tune the model on the training data and evaluate it on the validation data.

In [ ]:
# Set training args (just using defaults from the following tutorial for now:
# https://huggingface.co/docs/transformers/training )
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

# setup the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# train the model
trainer.train()

## Evaluate the model on the test set

In [ ]:
# evaluate on the test set
# should only do for _best_ model of each type 
# after selecting hyperparameters that work best on validation set
trainer.evaluate(tokenized_datasets["test"])

## Save the fine-tuned model and the tokenizer

Save the model and the tokenizer to a repository. To upload the model to Huggingface, you can download the tokenizer and model files to local storage > use git-lfs to push the files to their servers or using Huggingface GUI to upload files manually. Depending on your model, there should be at least 6 files.

- config.json
- pytorch_model.bin
- special_tokens_map.json
- tokenizer.json
- tokenizer_config.json
- vocab.txt

Additional files include

- optimizer.pt
- rng_state.pth
- scheduler.pt
- trainer_state.json
- training_args.bin


In [ ]:
model.save_pretrained("./the-path/where-you-save/")
tokenizer.save_pretrained("./the-path/you-want-to-save/")

# Using Classical Machine Learning Models to detect for the presence of narratives

## Import necessary libraries

In [ ]:
import collections
import csv
import os
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
#download any other libraries if it throws an error.

## Loading the Dataset

In [ ]:
## loading the data frame 

Corpus = pd.read_csv("Corpus.csv", encoding = "latin-1", sep = '\t')

# HAd to manually change it to str because pandas has a weird parsing error while vectorizing 
Corpus.text = Corpus.text.astype(str)

Corpus.info()

## Tokenize, POS Tag, Lemmatize, LowerCase, DropNA, StopWordRemoval

In [ ]:
#Convert the fields to lower case, so we maintain consistency between 'ant','Ant', 'ANT' etc
Corpus['text'] = Corpus['text'].str.lower()

#drop any null, blank fields Or else we wouldnt' be able to Lemmatize them below.
Corpus = Corpus.dropna()

Corpus.head()

The code snippet below is part of a text preprocessing pipeline used in natural language processing tasks. Here's a step-by-step breakdown of what it does:

- Tokenization:
    - The text in the 'text' column of the Corpus DataFrame is tokenized using the *nltk.word_tokenize* method, which splits text into individual words (tokens).
- Custom Stop Words:
    - A list named omit is created, containing custom-defined stop words.If you consider any words that are irrelevant to the analysis or model training, include them in the list and omit them in the process.
- Part-of-Speech (POS) Tagging Setup:
    - A defaultdict is used to create a 'tag_map' that maps the first letter of POS tags to corresponding word types recognized by the WordNet lemmatizer. This helps in lemmatizing words based on their POS tags:
        - 'J' for adjectives
        - 'V' for verbs
        - 'R' for adverbs
    - By default, any POS tag that does not start with 'J', 'V', or 'R' is assumed to be a noun.
- Lemmatization:
    - The code then iterates over the 'text' column of the Corpus DataFrame.
    - *Final_words* is initialized as an empty list to hold the processed words for each document.
    - *WordNetLemmatizer* is instantiated to lemmatize the words.

In [ ]:
#tokenizing our Narrative text column here
Corpus['text'] = Corpus['text'].apply(nltk.word_tokenize)

omit = []
# POS Tagging

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

#Lemmatize
for index,entry in enumerate(Corpus['text']):
    # empty list which I will append to the df in the end.
    Final_words = []
    
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        # check for Stop words and consider only alphabets
        if word not in list(set(stopwords.words('english'))-set(omit)) and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

## Splitting the Data, Encoding Labels, Tfidf Vectorizer

In [ ]:
#Train, test split
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],
                                                                    Corpus['label'],test_size=0.1, 
                                                                    random_state= 1000)

This part of the code below is concerned with the preparation of the label data for training and testing a machine learning model and the transformation of text data into a numerical format using TF-IDF (Term Frequency-Inverse Document Frequency) vectorization.

- Label Encoding:
    - The LabelEncoder from the sklearn.preprocessing module is used to encode the target labels (Train_Y and Test_Y), which could be categorical strings, into a numerical format. This is important because most machine learning algorithms require the target to be a numerical array.
    - fit_transform method is used to fit the label encoder to the training data and then transform it. The same is done for the test data. It should be noted that it is unusual to fit the encoder separately on both the training and test sets because this can lead to inconsistencies if there are labels in the test set that weren't seen during training. Usually, the encoder is fitted on the training data and then used to transform both the training and test sets.
- TF-IDF Vectorization:
    - A TfidfVectorizer is instantiated from sklearn.feature_extraction.text, which is used to convert a collection of raw text documents into a matrix of TF-IDF features.
    - fit method is called on the 'text_final' column of the Corpus DataFrame to learn the vocabulary and idf (inverse document frequency) from the training data.
    - transform method is then used to convert the Train_X and Test_X text data into the corresponding numerical matrix. This method uses the vocabulary and idf learned during the fit to perform the transformation.
    - The output Train_X_Tfidf and Test_X_Tfidf are sparse matrices representing the TF-IDF encoded text data, which can now be used as input features for training machine learning models.

In [ ]:
#Encoding our labels
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

#Vectorizer
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
# fit the NB classifier
Naive = naive_bayes.MultinomialNB()
naive_model = Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

In [ ]:
print(f1_score(predictions_NB, Test_Y),precision_score(predictions_NB, Test_Y),
      recall_score(predictions_NB, Test_Y))

In [ ]:
#SVM classifier
SVM = svm.SVC(C=2, kernel='poly',degree=2, gamma='scale', probability=True)
svm_model = SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
print(f1_score(predictions_SVM, Test_Y),precision_score(predictions_SVM, Test_Y),
      recall_score(predictions_SVM, Test_Y))

# Generative Models to Predict Narrativity in the tweets

In [ ]:
import collections
import csv
import os
import re
import pandas as pd
# imports - 3rd party
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
import pandas as pd
ds = pd.read_csv("data.csv", sep = ',')
full_df = pd.read_csv("data2.csv", sep = ",")

In [ ]:
y_true = ds["label"].tolist()

In [ ]:
import os
import openai

OPENAI_API_KEY = "your-api-key"
openai.api_key = OPENAI_API_KEY

In [ ]:
fs_d_i_n10_r1 =[]
fs_d_i_n10_l1 =[]

In [ ]:
prompt = "Definitions:\n(1)According to Kreuter et al.(2007), a narrative is defined as a representation of connected events and characters that has an identifiable structure, is bounded in space and time, and contains implicit or explicit messages about the topic being addressed.\n(2)Bilandzic & Busselle (2013) say that a narrative refers to a presentation of an event(s) experienced by specific character(s) in a setting.\n(3)And according to Dahlsrom (2021), a narrative is defined as a message that describes the experience of specific characters across a series of related events over a defined time period—a triumvirate of character, causality, and temporality. At its core, narrative is the telling of someone’s experience about something.\nThe rules for labeling the tweets are as follows: the tweet must contain (1) At least one specific character (normally is a person) who experiences (2) a series of related events.\nTweet: "+sampled_list[0]+ "\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[1]+ "\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[2]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[3]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[4]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[5]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[6]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[7]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[8]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[9]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[10]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[11]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[12]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[13]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[14]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[15]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[16]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[17]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[18]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[19]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[20]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[21]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[22]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[23]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[24]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[25]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[26]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[27]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[28]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[29]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[30]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[31]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[32]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[33]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[34]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[35]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[36]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[37]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[38]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[39]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+i+"\nQ: Is this tweet a narrative? Answer only Yes or No\nA:"

In [ ]:
start_sequence = "\nA:"
restart_sequence = "\n\nQ: "

for i in ds["text"][0:100]:
    sampled_df = full_df.groupby(['dataset', 'is_narrative']).sample(10)
    sampled_list = []
    for j in range(len(sampled_df)):
        sampled_list.append(sampled_df["tweet_text"].iloc[j])
    
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Definitions:\n(1)According to Kreuter et al.(2007), a narrative is defined as a representation of connected events and characters that has an identifiable structure, is bounded in space and time, and contains implicit or explicit messages about the topic being addressed.\n(2)Bilandzic & Busselle (2013) say that a narrative refers to a presentation of an event(s) experienced by specific character(s) in a setting.\n(3)And according to Dahlsrom (2021), a narrative is defined as a message that describes the experience of specific characters across a series of related events over a defined time period—a triumvirate of character, causality, and temporality. At its core, narrative is the telling of someone’s experience about something.\nThe rules for labeling the tweets are as follows: the tweet must contain (1) At least one specific character (normally is a person) who experiences (2) a series of related events.\nTweet: "+sampled_list[0]+ "\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[1]+ "\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[2]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[3]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[4]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[5]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[6]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[7]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[8]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[9]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[10]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[11]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[12]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[13]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[14]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[15]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[16]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[17]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[18]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[19]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[20]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[21]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[22]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[23]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[24]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[25]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[26]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[27]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[28]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[29]+"\nQ: Is this tweet a narrative?\nA: No\nTweet: "+sampled_list[30]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[31]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[32]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[33]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[34]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[35]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[36]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[37]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[38]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+sampled_list[39]+"\nQ: Is this tweet a narrative?\nA: Yes\nTweet: "+i+"\nQ: Is this tweet a narrative? Answer only Yes or No\nA:",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
  )
    fs_d_i_n10_r1.append(1 if response["choices"][0]["text"]==" Yes" else 0)

In [ ]:
fs_d_i_n10_l1.append(response["choices"][0]["text"])
   
    if response["choices"][0]["text"]== "No":
        fs_d_i_n10_r1.append(0)
    elif response["choices"][0]["text"]== "No.":
        fs_d_i_n10_r1.append(0)
    elif response["choices"][0]["text"]== "Yes":
        fs_d_i_n10_r1.append(1)
    elif response["choices"][0]["text"]== "Yes.":
        fs_d_i_n10_r1.append(1)
    else:
        fs_d_i_n10_r1.append(-1)

In [ ]:
import sklearn.metrics as metrics

print(metrics.confusion_matrix(y_true[0:200], fs_d_i_n10_r1[0:200]))
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true[0:200], fs_d_i_n10_r1[0:200], digits=3))